<a href="https://colab.research.google.com/github/Anushakadlimatti/Farmery/blob/main/Disease_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from collections import Counter

In [9]:
disease_data = pd.read_csv('/content/final_dataset.csv')

In [10]:
column_names = [col for col in disease_data.columns if col!='Disease']

In [11]:
columns_to_check = []
for col in disease_data.columns:
    if col != 'Disease':
        columns_to_check.append(col)

In [13]:
symptoms = disease_data.iloc[:,1].values.flatten()
symptoms = list(set(symptoms))

In [14]:
new_disease_data= disease_data.copy()
for symptom in symptoms:
  new_disease_data[symptom]=new_disease_data.iloc[:,1:].apply(lambda row: int(symptom in row.values),axis=1)

new_disease_data = new_disease_data.drop(columns=columns_to_check)

In [15]:
new_disease_data

,Disease,cramps,weight_gain,acidity,continuous_sneezing,neck_pain,indigestion,chest_pain,skin_rash,yellowish_skin,...,constipation,stomach_pain,vomiting,sunken_eyes,breathlessness,mood_swings,pain_during_bowel_movements,high_fever,stiff_neck,weakness_in_limbs
0,Fungal infection,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Fungal infection,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,Fungal infection,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fungal infection,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Fungal infection,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,(vertigo) Paroymsal Positional Vertigo,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4916,Acne,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4917,Urinary tract infection,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4918,Psoriasis,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
disease_data_1 = new_disease_data.loc[:, new_disease_data.columns.notna()]

In [17]:
disease_data_1.columns = disease_data_1.columns.str.strip()

In [18]:
target_variable = ['Disease']
label = LabelEncoder()
for i in target_variable:
  disease_data_1[i] = label.fit_transform(disease_data_1[i])

In [19]:
train_data = disease_data_1.drop(columns='Disease')
target_data = disease_data_1['Disease']

In [20]:
!pip install catboost
!pip install xgboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier  # Import XGBoost classifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
import numpy as np

algorithms = {
    'Logistic Regression': {"model": LogisticRegression()},
    'Random Forest': {"model": RandomForestClassifier()},
    'xGBoost': {"model": XGBClassifier()},

}

# Perform 5-fold cross-validation with specified metrics
for model_name, values in algorithms.items():
    model = values["model"]
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    accuracy_scores = []
    f1_scores = []
    recall_scores = []
    precision_scores = []

    for train_index, test_index in kf.split(train_data, target_data):
        X_train, X_test = train_data.iloc[train_index], train_data.iloc[test_index]
        y_train, y_test = target_data.iloc[train_index], target_data.iloc[test_index]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        precision = precision_score(y_test, y_pred, average='weighted')
        proba = model.predict_proba(X_test)[:, 1]


        accuracy_scores.append(accuracy)
        f1_scores.append(f1)
        recall_scores.append(recall)
        precision_scores.append(precision)


    print(f"{model_name} Results:")
    print(f"Accuracy: {np.mean(accuracy_scores):.3f}")
    print(f"F1 Score: {np.mean(f1_scores):.3f}")
    print(f"Recall: {np.mean(recall_scores):.3f}")
    print(f"Precision: {np.mean(precision_scores):.3f}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.9 MB/s eta 0:00:00
Logistic Regression Results:
Accuracy: 0.947
F1 Score: 0.941
Recall: 0.947
Precision: 0.942
Random Forest Results:
Accuracy: 0.947
F1 Score: 0.941
Recall: 0.947
Precision: 0.941
xGBoost Results:
Accuracy: 0.947
F1 Score: 0.941
Recall: 0.947
Precision: 0.944


In [21]:
doctor_data = pd.read_csv('/content/Disease_doctors.csv',encoding='latin1', names=['Disease','Specialist'])

In [22]:
test_column =[]
for col in disease_data_1.columns:
  if col!='Disease':
    test_column.append(col)

test_data = {}
symptoms = []
predicted = []
def test_input():
  symptoms.clear()
  predicted.clear()
  num = int(input('enter number of symptoms u have'))
  for i in range(num):
    userinput = input('Enter symptoms #{}:'.format(i+1))
    symptoms.append(userinput)
  print("Symptoms u have:" , symptoms)
  for column in test_column:
    test_data[column] = 1 if column in symptoms else 0
    test_df = pd.DataFrame(test_data,index=[0])
  print("Prediction in process")
  for model_name, values in algorithms.items():
    predict_disease = values["model"].predict(test_df)
    predict_disease = label.inverse_transform(predict_disease)
    predicted.extend(predict_disease)

  disease_counts = Counter(predicted)
  percentage_per_disease = {disease: (count/6) *100 for disease, count in disease_counts.items() }
  result_df =pd.DataFrame({"Disease":list(percentage_per_disease.keys()),"chances": list(percentage_per_disease.values())})
  result_df = result_df.merge(doctor_data, on='Disease', how='left')
  return result_df

In [26]:
test_input()

enter number of symptoms u have1
Enter symptoms #1:high_fever
Symptoms u have: ['high_fever']
Prediction in process


,Disease,chances,Specialist
0,Impetigo,50.0,Dermatologist
